# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

/Users/ghost/src/virtual_env/jupyther3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/ghost/src/virtual_env/jupyther3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=postgres")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
        
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath_song = song_files[0]
filepath_song

'/Users/ghost/src/github/jupyterlab/postgres/dataModel/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json'

In [6]:
df = pd.read_json(filepath_song, lines=True)

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id','title', 'artist_id','year','duration']].values.tolist()[0]
song_data

['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
try:
    cur.execute(song_table_insert, song_data)
    conn.commit()
except psycopg2.Error as e:
    cur.execute("ROLLBACK")
    print("Error: Inserting Rows",e)

Error: Inserting Rows duplicate key value violates unique constraint "songs_pkey"
DETAIL:  Key (song_id)=(SOCIWDW12A8C13D406) already exists.



Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id',
                  'artist_name', 
                  'artist_location',
                  'artist_latitude',
                  'artist_longitude']].values.tolist()[0]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
try:
    cur.execute(artist_table_insert, artist_data)
    conn.commit()
except psycopg2.Error as e:
    cur.execute("ROLLBACK")
    print("Error: Inserting Rows",e)

Error: Inserting Rows duplicate key value violates unique constraint "artists_pkey"
DETAIL:  Key (artist_id)=(ARMJAGH1187FB546F3) already exists.



Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

In [12]:
filepath_events = log_files[0]

In [13]:
df = pd.read_json(filepath_events, lines=True)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df.loc[df['page'] == 'NextSong']

In [15]:
t = pd.to_datetime(df['ts'], unit='ms')

In [16]:
time_data = (pd.DataFrame([
    df['ts'],
    t.dt.hour, 
    t.dt.day, 
    t.dt.week, 
    t.dt.month, 
    t.dt.year, 
    t.dt.weekday_name])).T
column_labels = (
    'timestamp',
    'hour', 
    'day', 
    'week of year', 
    'month', 
    'year',
    'weekday')

time_data.columns = column_labels
time_df = time_data

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [17]:
try:
    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))
        conn.commit()
except psycopg2.Error as e:
    cur.execute("ROLLBACK")
    print("Error: Inserting Rows",e)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [18]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [19]:
try:
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)
        conn.commit()
except psycopg2.Error as e:
    cur.execute("ROLLBACK")
    print("Error: Inserting Rows", e)

Error: Inserting Rows duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(8) already exists.



Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [20]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    
    try:
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
    except psycopg2.Error as e:
        cur.execute("ROLLBACK")
        print("Error: Inserting Rows",e)

Error: Inserting Rows insert or update on table "songplays" violates foreign key constraint "songplays_user_id_fkey"
DETAIL:  Key (user_id)=(10) is not present in table "users".

Error: Inserting Rows insert or update on table "songplays" violates foreign key constraint "songplays_user_id_fkey"
DETAIL:  Key (user_id)=(26) is not present in table "users".

Error: Inserting Rows insert or update on table "songplays" violates foreign key constraint "songplays_user_id_fkey"
DETAIL:  Key (user_id)=(26) is not present in table "users".

Error: Inserting Rows insert or update on table "songplays" violates foreign key constraint "songplays_user_id_fkey"
DETAIL:  Key (user_id)=(101) is not present in table "users".



## REMEMBER: Restart this notebook to close connection to `sparkifydb`
Each time you run the cells above, remember to restart this notebook to close the connection to your database. Otherwise, you won't be able to run your code in `create_tables.py`, `etl.py`, or `etl.ipynb` files since you can't make multiple connections to the same database (in this case, sparkifydb).

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [21]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.